In [1]:
import ROOT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ROOT.ROOT.EnableImplicitMT()

if "/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/func_marti.so" not in ROOT.gSystem.GetLibraries():
    ROOT.gSystem.CompileMacro("/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/func_marti.cc","k")

Welcome to JupyROOT 6.28/00


In [2]:
import ROOT
from os import path
import pandas as pd

ROOT.EnableImplicitMT()

def inspect_file (fileName):
    """Function for inspecting contents of a file.

    Args:
        fileName (str): Name of the ROOT file.
    """
    chain = ROOT.TChain('Events')
    chain.Add(fileName)

    cols = ['rho_gen_pt',
               'rho_gen_pdgId',
               'rho_gen_trk1_pt',
               'rho_gen_trk1_pdgId']

    df = ROOT.RDataFrame(chain)

    x = df.AsNumpy(columns=cols)
    pd_df = pd.DataFrame(x)
    display(pd_df)

if __name__=='__main__':
    dirName = '/data/submit/cms/store/user/mariadlf/nano/D01/'
    folderName = 'GluGlu_HToRhoGamma_M125_TuneCP5_PSWeights_13TeV_powheg_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v3+MINIAODSIM'
    fileName = '0225B464-D2E2-4348-B460-0275BCDD58D2.root'
    inspect_file(path.join(dirName, folderName, fileName))

,rho_gen_pt,rho_gen_pdgId,rho_gen_trk1_pt,rho_gen_trk1_pdgId
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.23681640625, 1.3408203125, 1.3408203125, 1....","[22, -211, -211, 321, -321, -211, -211, -211, ..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 9.956682205200195, 1...","[0, 0, 0, 0, 0, 411, 333, 0, 0, 0, 0, 0, 0, 0,...","[0.0, 1.890625, 1.890625, 0.0, 2.029296875, 6....","[0, -321, -321, 0, -211, 211, 321, 211, 0, 0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 2.076171875, 1.0986328125, 1.0986328125,...","[0, 211, -211, -211, 321, -211, -211, 211, 211..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 47.49160385131836, 0...","[0, 0, 0, 0, 0, 113, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1.076171875, 2.2890625, 1.265625, 1.357421875...","[211, -2212, 321, -211, -211, -211, 211, 2212,..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.3232421875, 1.5703125, 1.451171875, 1.57031...","[22, 211, 211, 211, 0, 0, 211, 22, 0, 0]"
...,...,...,...,...
53995,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 113, 0, 0, 0, 0...","[0.399658203125, 0.399658203125, 0.0, 2.150390...","[211, 211, 0, 211, 211, -211, 2212, -321, 2212..."
53996,"[0.0, 0.0, 0.0, 0.0, 0.0, 39.90202331542969, 0...","[0, 0, 0, 0, 0, 113, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1.2978515625, 1.5498046875, 1.2822265625, 1.2...","[-211, 321, -211, 211, 22, -211, 211, 211, 211..."
53997,"[0.0, 0.0, 53.857173919677734, 0.0, 0.0, 0.0, ...","[0, 0, 113, 0, 0, 0, 0]","[1.1083984375, 1.5634765625, 2.98046875, 2.658...","[-211, -211, -211, 211, -211, 321, 0]"
53998,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.0087890625, 1.4384765625, 3.51953125, 1.076...","[-211, -211, 211, 211, -211, -211, -211, 211, ..."


Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available
